### Test data from genderizeR project

In [1]:
import pandas as pd
import csv

In [5]:
test_data_raw = pd.read_csv('authorships_genderizeR_paper.csv')

In [40]:
test_data_raw.head()

,Unnamed: 0,WOSaccessionNumber,title,authors,value,genderCoded
0,1,A1993LL08800024,"MCNUTT,JAMES,WESLEY - OBITUARIES","ARMSON, KA","ARMSON, KA",noname
1,2,000307797400050,GLENN PRESTWICH,"Thayer, Ann","Thayer, Ann",unknown
2,3,A19668808300004,"OBITUARY - HEVESY,GV","ERDOS, J","ERDOS, J",noname
3,4,A1994PL30300031,NIXON AND THE DISABILITY-RIGHTS MOVEMENT .2. A...,"LONGMORE, P","LONGMORE, P",noname
4,5,A1993KU74400007,"ABEL,YVES",Anonymous,Anonymous,noname


In [41]:
test_data_raw[test_data_raw.genderCoded != 'noname']

,Unnamed: 0,WOSaccessionNumber,title,authors,value,genderCoded
1,2,000307797400050,GLENN PRESTWICH,"Thayer, Ann","Thayer, Ann",unknown
11,12,000263489500011,Giovanni Orlandi,"Chiesa, Paolo","Chiesa, Paolo",male
24,25,000273316000001,IN MEMORY OF PIETRO PASSERINI,"Abbate, Ernesto","Abbate, Ernesto",male
28,29,000245112000038,"William L. Epstein, 1925-2006 - In memoriam","Epstein, John H.","Epstein, John H.",male
34,35,000246578700013,In remembrance of Ivan Boszormenyi-Nagy MD,"Cotroneo, Margaret","Cotroneo, Margaret",female
38,39,000250625400001,The structure of Bence-Jones proteins: the wor...,"Kresge, Nicole;Simoni, Robert D.;Hill, Robert L.","Kresge, Nicole",female
39,40,000250625400001,The structure of Bence-Jones proteins: the wor...,"Kresge, Nicole;Simoni, Robert D.;Hill, Robert L.","Simoni, Robert D.",male
40,41,000250625400001,The structure of Bence-Jones proteins: the wor...,"Kresge, Nicole;Simoni, Robert D.;Hill, Robert L.","Hill, Robert L.",male
59,60,000270370900015,Thomas L Dao Obituary,"Williams, Ruth","Williams, Ruth",female
61,62,000256494100030,A.R Gokul (1924-2008) - Obituary,"Srikantia, S. V.;Rao, V. R. Venkoba","Rao, V. R. Venkoba",unknown


In [11]:
test_data_raw.columns

Index(['Unnamed: 0', 'WOSaccessionNumber', 'title', 'authors', 'value',
       'genderCoded'],
      dtype='object')

In [33]:
# Extract the data in the same format than the zbMATH test data
# columns: id, raw_name, first_name, middle_name, last_name, gender

def split_name(raw_name):
    # Lower case
    raw_name = raw_name.lower()
    # Everything before a comma is last_name
    try:
        last_name, name = raw_name.strip().strip('.').split(',')
    except ValueError:
        name = None
        last_name, first_name, middle_name = raw_name.strip().strip('.'), None, None
    # Now parse name
    if name:
        try:
            first_name, middle_name = name.strip().strip('.').split()
        except ValueError: 
            first_name, middle_name = name.strip().strip('.'), None
    return first_name, middle_name, last_name 

In [17]:
gender_keys = {'noname': 'noname', 
               'unknown': 'u', 
               'male': 'm',
               'female': 'f'}

In [34]:
test_data = pd.DataFrame()

In [35]:
test_data['id'] = test_data_raw['Unnamed: 0']
test_data['raw_name'] = test_data_raw['value']
test_data['first_name'], test_data['middle_name'], test_data['last_name'] = zip(*test_data_raw['value'].map(split_name))
test_data['gender'] = test_data_raw['genderCoded'].apply(lambda n: gender_keys[n])

In [36]:
test_data

,id,raw_name,first_name,middle_name,last_name,gender
0,1,"ARMSON, KA",ka,None,armson,noname
1,2,"Thayer, Ann",ann,None,thayer,u
2,3,"ERDOS, J",j,None,erdos,noname
3,4,"LONGMORE, P",p,None,longmore,noname
4,5,Anonymous,None,None,anonymous,noname
5,6,"Shavelson, RJ",rj,None,shavelson,noname
6,7,"Gleser, G",g,None,gleser,noname
7,8,"Kagarise, MJ",mj,None,kagarise,noname
8,9,"Sheldon, GF",gf,None,sheldon,noname
9,10,"Krimigis, SM",sm,None,krimigis,noname


In [38]:
test_data[test_data.gender != 'noname']

,id,raw_name,first_name,middle_name,last_name,gender
1,2,"Thayer, Ann",ann,None,thayer,u
11,12,"Chiesa, Paolo",paolo,None,chiesa,m
24,25,"Abbate, Ernesto",ernesto,None,abbate,m
28,29,"Epstein, John H.",john,h,epstein,m
34,35,"Cotroneo, Margaret",margaret,None,cotroneo,f
38,39,"Kresge, Nicole",nicole,None,kresge,f
39,40,"Simoni, Robert D.",robert,d,simoni,m
40,41,"Hill, Robert L.",robert,l,hill,m
59,60,"Williams, Ruth",ruth,None,williams,f
61,62,"Rao, V. R. Venkoba",v. r. venkoba,None,rao,u
